# Plot a histogram of TMA tracking jitter from RubinTV.
Craig Lage - 22-Apr-24


# Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
dataDir = Path("./plots")
dataDir.mkdir(exist_ok=True, parents=True)

# These are the nights when the tests were run
firstDayObs = 20240409
lastDayObs = 20240421

In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
%matplotlib inline

### This uses the json metadata from RubinTV, which contains the astrometric solutions for the StarTracker exposures.  You need to download the metadata for each dayObs and put it in a location where the program can find it.

In [ ]:
rmsError = []
dayObs = firstDayObs
while dayObs <= lastDayObs:
    try:
        dayObsString = dayObsIntToString(dayObs)
        filename = f'../json_metadata/tma_{dayObsString}.json'
        jsonData = pd.read_json(filename).T
        jsonData = jsonData.sort_index()
        values = jsonData['Tracking image impact'].values
        for value in values:
            if not np.isnan(float(value)):
                rmsError.append(float(value))
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        continue
print(f" There are {len(rmsError)} valid tracking events.")        

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,1,figsize=(5,5))
rmsError = np.array(rmsError)
goodRms = rmsError[rmsError < 0.01]
percentGoodRms = len(goodRms) / len(rmsError) * 100.0
rmsMed = np.nanmedian(rmsError)
ax.set_title(f"Tracking jitter {firstDayObs}-{lastDayObs}.\nN={len(rmsError)}, Median={rmsMed:.3f}, Less than 0.01 = {percentGoodRms:.1f} %")
ax.hist(rmsError, range=(0,0.1), bins=100)
ax.axvline(0.01, color='black', ls = '--')
ax.set_xlim(0,0.05)
ax.set_xlabel("RMS Jitter (arcseconds)")
plt.savefig(str(dataDir / f"Jitter_Summary_{firstDayObs}-{lastDayObs}.png"))